In [1]:
import pandas as pd
import numpy as np
import requests
from scipy import stats

In [2]:
api_url = 'https://sandbox.iexapis.com/stable/'

with open('../secret.txt') as f:
    token = f.readline()
token = 'token=' + token

In [3]:
with open('sp_500_stocks.csv') as f:
    stocklist =  f.readlines()[1:]
stocklist = [s[:-1] for s in stocklist]

In [4]:
def get_batch(string, types='quote,advanced-stats'):
    url = api_url + f'/stock/market/batch?symbols={string}&types={types}&' + token
    D = requests.get(url).json()
    L = []
    for res in D.values():
        d = {}
        for types in res.values():
            d.update(types)
        L.append(d)
        
    return pd.DataFrame(L)

def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [9]:
my_columns = [
    'symbol',
    'latestPrice'
    'marketCap',
    'priceToBook',
    'profitMargin',
    'revenue'
]

In [6]:
DFb = pd.DataFrame()
for sublist in chunks(stocklist, 100):
    symbol = ','.join(sublist)
    DFb = DFb.append(get_batch(symbol), ignore_index=True)
    
DFb.head()

,beta,totalCash,currentDebt,revenue,grossProfit,totalRevenue,EBITDA,revenuePerShare,revenuePerEmployee,debtToEquity,...,iexOpen,iexOpenTime,iexClose,iexCloseTime,marketCap,week52High,week52Low,ytdChange,lastTradeTime,isUSMarketOpen
0,NaN,1.342693e+09,1.725119e+09,5.691122e+09,2.966814e+09,5.556725e+09,1.323100e+09,18.88,420811.90,2.112715,...,130.52,1.670236e+12,131.420,1664154216482,40221320190,139.12,80.55,0.092333,1682006725721,True
1,NaN,1.473408e+10,1.785280e+10,1.313137e+10,1.299492e+10,1.298726e+10,-1.096147e+10,20.00,96958.51,-9.036204,...,23.76,1.621754e+12,23.177,1660646813755,14738367831,27.37,9.70,0.481345,1643855405291,True
2,NaN,8.402953e+08,4.917155e+09,1.012795e+10,4.672954e+09,1.049264e+10,1.037794e+09,158.55,151234.71,3.382132,...,198.27,1.700845e+12,201.540,1660658749395,13133516118,215.12,132.37,0.250219,1650310064807,True
3,NaN,6.997453e+10,1.088051e+11,3.330335e+11,1.346077e+11,3.384960e+11,1.047334e+11,19.50,2300486.67,4.902905,...,125.17,1.699460e+12,126.317,1696124105481,2110400168844,147.86,78.89,-0.067517,1625320229944,True
4,0.63161,1.001848e+10,3.217598e+10,5.211946e+10,3.314585e+10,5.046682e+10,2.218297e+10,29.32,1695177.37,11.491952,...,119.60,1.659688e+12,119.700,1695517792500,203674054508,117.80,79.39,0.095731,1700980521643,True


In [44]:
DF = DFb[my_columns]

In [45]:
DF.head()

,symbol,marketCap,priceToBook,profitMargin,revenue
0,A,40221320190,8.35,0.148565,5.691122e+09
1,AAL,14738367831,-1.90,-0.642044,1.313137e+10
2,AAP,13133516118,3.77,0.051181,1.012795e+10
3,AAPL,2110400168844,30.24,0.234777,3.330335e+11
4,ABBV,203674054508,15.30,0.105562,5.211946e+10


In [46]:
DF['bookToPrice'] = 1/DF.priceToBook
DF['BookValue'] = DF.marketCap * DF['bookToPrice']

c:\users\lazyboy\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\users\lazyboy\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [47]:
DF.head()

,symbol,marketCap,priceToBook,profitMargin,revenue,bookToPrice,BookValue
0,A,40221320190,8.35,0.148565,5.691122e+09,0.119760,4.816925e+09
1,AAL,14738367831,-1.90,-0.642044,1.313137e+10,-0.526316,-7.757036e+09
2,AAP,13133516118,3.77,0.051181,1.012795e+10,0.265252,3.483691e+09
3,AAPL,2110400168844,30.24,0.234777,3.330335e+11,0.033069,6.978837e+10
4,ABBV,203674054508,15.30,0.105562,5.211946e+10,0.065359,1.331203e+10


In [48]:
def mx0(n):
    return n if n>0 else 0

### Strategy 1: Book value weighted

In [49]:
DF['s1'] = DF.BookValue.apply(mx0)
DF.s1 /= (DF.s1.sum()/100)

c:\users\lazyboy\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\users\lazyboy\appdata\local\programs\python\python37\lib\site-packages\pandas\core\generic.py:5165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


### Strategy 2: root of Book value weighted

In [50]:
DF['s2'] = DF.BookValue.apply(mx0)
DF.s2 = DF.s2**0.5
DF.s2 /= (DF.s2.sum()/100)

c:\users\lazyboy\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Strategy 3: 2**bookToPrice-1.5 weighted

In [71]:
DF['s3'] = 2**DF.bookToPrice-1.5
DF.s3 = DF.s3.apply(mx0)
DF.s3 /= (DF.s3.sum()/100)

c:\users\lazyboy\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\users\lazyboy\appdata\local\programs\python\python37\lib\site-packages\pandas\core\generic.py:5165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [72]:
DF.head()

,symbol,marketCap,priceToBook,profitMargin,revenue,bookToPrice,BookValue,s1,s2,s3
0,A,40221320190,8.35,0.148565,5.691122e+09,0.119760,4.816925e+09,0.062017,0.141677,0.0
1,AAL,14738367831,-1.90,-0.642044,1.313137e+10,-0.526316,-7.757036e+09,0.000000,0.000000,0.0
2,AAP,13133516118,3.77,0.051181,1.012795e+10,0.265252,3.483691e+09,0.044852,0.120485,0.0
3,AAPL,2110400168844,30.24,0.234777,3.330335e+11,0.033069,6.978837e+10,0.898513,0.539270,0.0
4,ABBV,203674054508,15.30,0.105562,5.211946e+10,0.065359,1.331203e+10,0.171390,0.235525,0.0


In [73]:
DF[DF.s3 > 0]

,symbol,marketCap,priceToBook,profitMargin,revenue,bookToPrice,BookValue,s1,s2,s3
17,AFL,38092845900,1.2100,0.239870,2.468867e+10,0.826446,3.148169e+10,0.405321,0.362196,1.525113
18,AIG,43358625959,0.7083,-0.090787,4.445993e+10,1.411831,6.121506e+10,0.788133,0.505061,6.477106
20,AIZ,9487798889,1.6900,0.044360,1.025542e+10,0.591716,5.614082e+09,0.072280,0.152952,0.039274
47,ATO,13153000221,1.6800,0.221936,3.298009e+09,0.595238,7.829167e+09,0.100799,0.180623,0.059829
56,BAC,365509453063,1.4700,0.232018,9.365659e+10,0.680272,2.486459e+11,3.201273,1.017900,0.571638
...,...,...,...,...,...,...,...,...,...,...
476,WAB,15291329318,1.4800,0.058266,7.757391e+09,0.675676,1.033198e+10,0.133022,0.207494,0.543195
482,WFC,194959072921,1.2200,0.078751,7.880292e+10,0.819672,1.598025e+11,2.057430,0.816030,1.478759
489,WRK,15740922000,1.4900,-0.041480,1.774422e+10,0.671141,1.056438e+10,0.136015,0.209815,0.515222
496,XOM,251072689899,1.6900,-0.104907,1.897685e+11,0.591716,1.485637e+11,1.912732,0.786812,0.039274


In [74]:
DF.describe()

,marketCap,priceToBook,profitMargin,revenue,bookToPrice,BookValue,s1,s2,s3
count,5.040000e+02,486.000000,487.000000,4.910000e+02,486.000000,4.860000e+02,504.000000,504.000000,504.000000
mean,7.671065e+10,-8.364430,-0.181021,2.536884e+10,0.293414,1.580193e+10,0.198413,0.198413,0.198413
std,1.908281e+11,274.421147,4.612276,5.079862e+10,0.271546,2.961233e+10,0.374773,0.157797,0.823668
min,9.818874e+08,-5736.360000,-97.115896,3.643830e+07,-0.526316,-1.713871e+10,0.000000,0.000000,0.000000
25%,1.719280e+10,1.975000,0.048314,4.614361e+09,0.099825,3.076080e+09,0.036911,0.109300,0.000000
50%,3.017670e+10,3.920000,0.114249,1.029767e+10,0.223215,6.777683e+09,0.080334,0.161248,0.000000
75%,6.080619e+10,7.517500,0.189708,2.120737e+10,0.436681,1.524512e+10,0.189065,0.247371,0.000000
max,2.110400e+12,275.990000,1.590778,5.823277e+11,1.679825,2.530977e+11,3.258589,1.026972,9.507920


In [75]:
DF.to_csv('simpleStrats.csv')